# Libraries

In [1]:
import os
import xarray as xr
import rioxarray as rxr

In [2]:
# Set main directory using an environment variable or fallback to a relative path
main_dir = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'FAPAR anomaly'))

# Directory for monthly FAPAR anomaly data
FAPAR_anomaly = os.path.join(main_dir, 'FAPAR Anomaly', 'RT1')
# Directory for crop mask files for each month
Crop_mask = os.path.join(main_dir, 'Crop_mask')

In [4]:
# Dictionary to store crop masks for each month
Crop_mask_month = {}
for image in os.listdir(Crop_mask):
    path = os.path.join(Crop_mask, image)
    crop_mask = rxr.open_rasterio(path, masked=True)
    Crop_mask_month[image[:-4]] = crop_mask

In [6]:
# Loop through each monthly FAPAR anomaly and apply crop mask
for image in os.listdir(FAPAR_anomaly):
    # Path for monthly FAPAR anomaly file
    path = os.path.join(FAPAR_anomaly, image)
    # Open monthly FAPAR anomaly file
    FAPAR_a = rxr.open_rasterio(path, masked=True)
    # Retrieve corresponding crop mask for the same month
    monthly_crop_mask = Crop_mask_month[image[:-22]]

    # Mask values based on the crop mask, setting non-matching areas to -9999
    FAPAR_anomaly_mask = xr.where(monthly_crop_mask == 1, FAPAR_a[0], -9999)
    # Convert -9999 values to NaN
    FAPAR_anomaly_mask = FAPAR_anomaly_mask.where(FAPAR_anomaly_mask != -9999)
    # Set CRS to match the original FAPAR anomaly
    FAPAR_anomaly_mask = FAPAR_anomaly_mask.rio.write_crs(FAPAR_a.rio.crs)

    # Export the masked FAPAR anomaly as a TIFF file
    output_path = os.path.join(main_dir, 'FAPAR Anomaly_crop_mask', 'RT1', f'{image[:-4]}_crop_mask.tif')
    FAPAR_anomaly_mask.rio.to_raster(output_path)